# Mini Project 2: IMDB Sentiment Analysis

February 22, 2019

Akshal Aniche, Jacob Sanz-Robinson, Raphael Hotter

COMP 551

## Imports

In [2]:
import os

## Load the data

In [3]:
pos_train_files = os.listdir('train/pos/')
neg_train_files = os.listdir('train/neg/')
test_files = os.listdir('test/')

# Remove .DS_Store files
while '.DS_Store' in pos_train_files:
  pos_train_files.remove('.DS_Store')
while '.DS_Store' in neg_train_files:
  neg_train_files.remove('.DS_Store')
while '.DS_Store' in test_files:
  test_files.remove('.DS_Store')

In [6]:
# Reads data into 3 arrays
pos_train_raw, neg_train_raw, test_raw = [], [], []

for file in pos_train_files:
  with open('data/train/pos/{}'.format(file), 'r') as f:
    pos_train_raw.append({'filename': file, 'text': f.read()})

for file in neg_train_files:
  with open('data/train/neg/{}'.format(file), 'r') as f:
    neg_train_raw.append({'filename': file, 'text': f.read()})

for file in test_files:
  with open('data/test/{}'.format(file), 'r') as f:
    test_raw.append({'filename': file, 'text': f.read()})

In [8]:
pos_train_raw[0]

{'filename': '1821_4',
 'text': "Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form."}